In [1]:
import numpy as np
from numba import njit, prange
import itertools as it

### Binary enumeration


The objective is to enumerate all of the possible combinations. To do this we  

In [2]:
#Compute all binary arrays of length n_x * n_y
def get_configurations(n_x,n_y):

    # Define the length of the binary numbers
    N = n_x * n_y

    # Define the number of binary numbers
    M = 2**N

    # Create an empty NumPy array of shape (N, M)
    binary_array = np.zeros((N, M), dtype='int32')

    # Fill each row with binary numbers
    for i in range(M):
        binary_representation = np.binary_repr(i, width=N)  # Get binary representation
        binary_array[:, i] = np.array(list(map(int, binary_representation)))  # Fill the row with binary values

    binary_array = binary_array.T
    return binary_array

n_x, n_y, n_z = 2,2,2

binary_array = get_configurations(n_x, n_y)

In [44]:
#find the bitwise and between each string and return a truth table where if X[i,j] = 1 then the ith and jth configurations are valid
# note that this only returns full occupation combinations
@njit(parallel = True)
def compute_truth_array(binary_array : np.array):
    #create empty array
    truth_array = np.zeros((binary_array.shape[0],binary_array.shape[0]))
    #parallel loop to find possible configurations
    for i in prange(binary_array.shape[0]):
        for j in range(i, binary_array.shape[0]):
            if (binary_array[i] & binary_array[j]).any() == 0:
                truth_array[i,j] = 1
            # truth_array[i,j] = np.sum(binary_array[i] & binary_array[j])
    return truth_array

truth_array = compute_truth_array(binary_array)

In [48]:
def get_possible_neighbours(x,y,z,n_x,n_y,n_z):
    neightbours = []
    z_hat = [z_ for z_ in range(n_z)]

    for z_ in z_hat:
        if x + 1 < n_x and x + 1 >= 0:
            neightbours.append((x + 1, y, z_))
        if x - 1 < n_x and x - 1 >= 0:
            neightbours.append((x - 1, y, z_))
        if y + 1 < n_y and y + 1 >= 0:
            neightbours.append((x, y + 1, z_))
        if y - 1 < n_y and y - 1 >= 0:
            neightbours.append((x, y - 1, z_))

    return neightbours

def possible_adj_matrix(c1,c2,n_x,n_y,n_z,K):
    c1,c2 = c1.reshape((n_x,n_y)),c2.reshape((n_x,n_y))
    X = np.zeros((n_x,n_y,n_z))
    X[:,:,0] = c1
    X[:,:,1] = c2
    A = np.zeros((n_x*n_y*n_z,n_x*n_y*n_z))
    for i in range(n_x):
        for j in range(n_y):
            for k in range(n_z):
                for i_,j_,k_ in get_possible_neighbours(i,j,k,n_x,n_y,n_z):
                    A[i + j*n_x + k * n_x**2, i_ + j_ * n_x + k_ * n_x**2] = X[i,j,k] * X[i_,j_,k_] * K[k,k_]
    return A

In [49]:
K = np.array([[1,2],[3,4]])
possible_adj_matrix(np.array([1,1,1,1]),np.array([1,1,1,1]),2,2,2,K)

array([[0., 1., 1., 0., 0., 2., 2., 0.],
       [1., 0., 0., 1., 2., 0., 0., 2.],
       [1., 0., 0., 1., 2., 0., 0., 2.],
       [0., 1., 1., 0., 0., 2., 2., 0.],
       [0., 3., 3., 0., 0., 4., 4., 0.],
       [3., 0., 0., 3., 4., 0., 0., 4.],
       [3., 0., 0., 3., 4., 0., 0., 4.],
       [0., 3., 3., 0., 0., 4., 4., 0.]])

In [51]:
adjacency_mats = {}
for i,c1 in enumerate(binary_array):
    for j,c2 in enumerate(binary_array):
        if truth_array[i,j] == 1:
            adjacency_mats.update({f'{c1},{c2}' : possible_adj_matrix(c1,c2,n_x,n_y,n_z,K)})
adjacency_mats

{'[0 0 0 0],[0 0 0 0]': array([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]),
 '[0 0 0 0],[0 0 0 1]': array([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]),
 '[0 0 0 0],[0 0 1 0]': array([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
   